In [1]:
# import library
import pandas as pd
import numpy as np
import nltk
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()  
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# read dataset and known shape 
# Data Acquisition
data = pd.read_csv("wiki_movie_plots_deduped.csv")
data.head()
data.shape

 # make the Origin/Ethnicity is special
np.unique(data['Origin/Ethnicity'])

 # Choose part of data
part_data= data.loc[(data['Origin/Ethnicity']=='American') & (data['Release Year']>2015)]
print(len(part_data))
 # data frame to make data in table 
my_data = pd.DataFrame(part_data) 
 # default show firt 5 row of data
my_data.head()                     

 #make the final data to test .. final_data is title and discreption of film
final_data=my_data[['Title','Plot']]    
 # make index in title column
final_data=final_data.set_index('Title') 


 # default show firt 5 row of data
final_data.head()                       
 # show discreption of Broken Flowers film for test 
final_data["Plot"]["Split"]     

350


'Casey Cooke is an emotionally withdrawn teenager, having been molested as a child by her uncle John, who became her legal guardian after her father\'s death. While hanging out with her classmates, Claire and Marcia, the three are kidnapped by Kevin Wendell Crumb, a man suffering from dissociative identity disorder. Kevin has been evaluated by psychologist Dr. Karen Fletcher, who discovered that he formed multiple personalities as a means to cope with the abuse he suffered at the hands of his mother. While Kevin appears otherwise stable, Dr. Fletcher has identified 23 distinct personalities within Kevin; in his mind, these personalities sit in chairs in a room, waiting for their turn "in the light" (controlling Kevin\'s body) as directed by "Barry", the dominant personality. She has also found that Kevin\'s physiology changes when a new personality comes into the light. Recently, "Barry" has refused to allow "Dennis" or "Patricia" their turns, in part due to Dennis\' tendencies towards

In [3]:
# preprocessing and clean up data
def preprocessing (data):
     # convert data to lower case
    data = data.lower()
    
     # tokenize data into words
    words = nltk.word_tokenize(data)
     # make a lemmatization to return words to base form
    lemmatizer = WordNetLemmatizer()  
     # remove the stop words from text too
    sent=[lemmatizer.lemmatize(word) for word in words 
          if word not in stopwords.words('english')]
     # add space between sentence
    final_sent = ' '.join(sent)   
    
     # replace an abbriviation to the base 
    final_sent = final_sent.replace("n't", " not")
    final_sent = final_sent.replace("'m", " am")
    final_sent = final_sent.replace("'s", " is")
    final_sent = final_sent.replace("'re", " are")
    final_sent = final_sent.replace("'ll", " will")
    final_sent = final_sent.replace("'ve", " have")
    final_sent = final_sent.replace("'d", " would")
    
    return final_sent
 # apllay the preprocessing function on text
final_data["new_plot"]= final_data["Plot"].apply(preprocessing)
 # display the head of data ( default is 5 record )
final_data.head()

,Plot,new_plot
Title,,
The Forest,The majority of the story is set in and around...,"majority story set around aokigahara forest , ..."
Anesthesia,Philosophy professor Walter Zarrow is wounded ...,philosophy professor walter zarrow wounded mug...
Lamb,The film opens with David Lamb (Ross Partridge...,film open david lamb ( ross partridge ) visiti...
Ride Along 2,The film starts in Miami on the boat of crime ...,film start miami boat crime lord antonio pope ...
13 Hours: The Secret Soldiers of Benghazi,"In 2012, Benghazi, Libya is named one of the m...","2012 , benghazi , libya named one dangerous pl..."


In [4]:

# TF_IDF Methode to determine similarity

# import the tf-idf from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
# get object of tf-idf vectorizer
tfidf = TfidfVectorizer()
# make a Training with fit_transform
tfidf_movieid = tfidf.fit_transform((final_data["new_plot"])) # make a Training with fit_transform
# import the cosine similarity from metrics to calc similarity
from sklearn.metrics.pairwise import cosine_similarity
# calculate similarity 
similarity = cosine_similarity(tfidf_movieid, tfidf_movieid)


In [74]:
# function to make movies recomendation
# get a list of index
indices = pd.Series(final_data.index)    
# based on name of film and similarity
def movies_recommendation(title, cosine_similarity = similarity): 
    # apply try if right or similarity
  try:
    # get the index of the title
    index = indices[indices == title].index[0]   
    #print(index)
    
    # calculate the similarity score
    similarity_scores = pd.Series(cosine_similarity[index]).sort_values(ascending = False) 
    rounded_Similarity=similarity_scores.round(3)
    
    #get the 5 films with same similarity
    top_5_movies = list(similarity_scores.iloc[0:5].index) 
    
    # print the similarity between films 
    print(f"the Index of films that has similarity = {top_5_movies}") # the index of films that has similarity 
    print(f"the similarity score of films that has similarity = {list(rounded_Similarity.iloc[0:5])}")
    
    # get list with 5 similarity films 
    #recommended_movies = [list(final_data.index)[i] for i in top_5_movies] 
    return pd.DataFrame({'Related Films':[list(final_data.index)[i] for i in top_5_movies]})


    # apply catch if right or similarity
  except:
    print("No Result Founded")  

    
    

In [75]:
# this function get one parameter (is the name of film)
movies_recommendation("Youth in Oregon")

the Index of films that has similarity = [153, 75, 141, 275, 122]
the similarity score of films that has similarity = [1.0, 0.074, 0.06, 0.048, 0.041]


,Related Films
0,Youth in Oregon
1,Free State of Jones
2,100 Streets
3,The Good Catholic
4,Max Steel


In [70]:
# this function get one parameter (is the name of film)
movies_recommendation("American Fable")

the Index of films that has similarity = [161, 211, 135, 148, 245]
the Index of films that has similarity = [1.0, 0.115, 0.099, 0.07, 0.053]


,Related Films
0,American Fable
1,Lowriders
2,23
3,A Dog's Purpose
4,A Ghost Story


In [76]:
# this function get one parameter (is the name of film)
movies_recommendation("The House")

the Index of films that has similarity = [241, 72, 306, 290, 193]
the similarity score of films that has similarity = [1.0, 0.111, 0.095, 0.083, 0.08]


,Related Films
0,The House
1,Central Intelligence
2,The Killing of a Sacred Deer
3,The Mountain Between Us
4,Gifted


In [77]:
# this function get one parameter (is the name of film)
movies_recommendation("John Wick: Chapter 2")

the Index of films that has similarity = [157, 77, 105, 32, 139]
the similarity score of films that has similarity = [1.0, 0.25, 0.238, 0.217, 0.12]


,Related Films
0,John Wick: Chapter 2
1,The Legend of Tarzan
2,When the Bough Breaks
3,"Hello, My Name Is Doris"
4,The Bye Bye Man
